In [52]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os

# custom imports (from current directory)
from run_numbers_vars import *  # contains dark, blank, Ne, SF6 run numbers

###
# FUNCTIONS
def is_file(path):
    return os.path.isfile(path)

def runNumToString(num):
    numstr = str(num)
    while len(numstr)<4:
        numstr = '0'+numstr
    return numstr
    
def is_leaf(dataset):
    return isinstance(dataset,h5py.Dataset)

def get_leaves(f,saveto,verbose=False):
    def return_leaf(name):
        if is_leaf(f[name]):
            if verbose:
                print(name,f[name][()].shape)
            saveto[name] = f[name][()]
    f.visit(return_leaf)

def combineRuns(runNumbers,folder,verbose=False):
    data_array = []
    for i,runNumber in enumerate(runNumbers):
        data = {}
        filename = f'{folder}cxilx9320_Run{runNumToString(runNumber)}.h5'
        if is_file(filename):
            print('%s, FILE EXISTS, CONTINUE...' % filename)
        else:
            print('%s, FILE DOES NOT EXIST!' % filename)
        with h5py.File(filename,'r') as f:
            get_leaves(f,data,verbose=verbose)
            data_array.append(data)
    hf1 = h5py.File(filename, 'r')
#    for key in hf1['epicsAll']:
#        print(key)
    data_combined = {}
    for key in keys_to_combine:
        arr = np.squeeze(data_array[0][key])
        for data in data_array[1:]:
            arr = np.concatenate((arr,np.squeeze(data[key])),axis=0)
        data_combined[key] = arr
    run_indicator = np.array([])
    for i,runNumber in enumerate(runNumbers):
        run_indicator = np.concatenate((run_indicator,runNumber*np.ones_like(data_array[i]['tt/FLTPOS'])))
    data_combined['run_indicator'] = run_indicator
    for key in keys_to_sum:
        arr = np.zeros_like(data_array[0][key])
        for data in data_array:
            arr += data[key]
        data_combined[key] = arr
    for key in keys_to_check:
        arr = data_array[0][key]
        for i,data in enumerate(data_array):
            if not np.array_equal(data[key],arr):
                print(f'Problem with key {key} in run {runNumbers[i]}')
        data_combined[key] = arr
    return data_combined
# END FUNCTIONS
###

# KEYS
keys_to_combine = ['tt/FLTPOS',
                   'tt/AMPL',
                   'tt/FLTPOSFWHM',
                   'tt/ttCorr',
                   'tt/FLTPOS_PS',
                   'tt/AMPL',
                   'tt/AMPLNXT',
                   'tt/FLTPOS',
                   'tt/FLTPOS_PS',
                   'tt/REFAMPL',
                   #'scan/lxt_ttc',
                   'ipm_dg2/sum',
                   'ipm_dg3/sum',
                   'gas_detector/f_11_ENRC',
                   #'epicsAll/gasCell_pressure',
                   'jungfrau4M/azav_azav',
                   'evr/code_183',
                   'evr/code_162',
                   'evr/code_141',
                   #'epicsAll/gasCell_pressure',
                   'lightStatus/laser',
                   'lightStatus/xray',
                   # 'scan/var0',
                   #  'scan/LAS:FS5:MMS:PH',
                  ]

keys_to_sum = ['Sums/jungfrau4M_calib',
              'Sums/jungfrau4M_calib_thresADU1']

keys_to_check = ['UserDataCfg/jungfrau4M/azav__azav_q',
                'UserDataCfg/jungfrau4M/azav__azav_qbin',
                'UserDataCfg/jungfrau4M/azav__azav_qbins',
                'UserDataCfg/jungfrau4M/x',
                'UserDataCfg/jungfrau4M/y',
                'UserDataCfg/jungfrau4M/z',
                'UserDataCfg/jungfrau4M/azav__azav_matrix_q',
                'UserDataCfg/jungfrau4M/azav__azav_matrix_phi',
                'UserDataCfg/jungfrau4M/cmask',
                #'UserDataCfg/jungfrau4M/Full_thres__Full_thres_thresADU',
                #'UserDataCfg/jungfrau4M/Full_thres__Full_thres_bound',
                'UserDataCfg/jungfrau4M/common_mode_pars']

# END KEYS
###

# THOMAS FUNCTIONS
def normalise(x, f):
    f *= abs(x[1] - x[0])
    f /= np.sum(f)
    return f

def get_azav(runNumbers):
    '''
    Returns normalised azimuthal average of Jungfrau for array of run numbers.
    '''
    data = combineRuns(runNumbers, folder=folder, verbose=False)
    # azimuthal average
    #azav = np.squeeze(data['jungfrau4M/azav_azav'])
    azav  = data['jungfrau4M/azav_azav']
    print('azav shape: '); print(azav.shape)
    #print(azav.shape)
    azav_sum = np.sum(azav, axis=0)
    
    # q definition
    q = data['UserDataCfg/jungfrau4M/azav__azav_q']
    #qbin  = data['UserDataCfg/jungfrau4M/azav__azav_qbin' ]
    #qbins = data['UserDataCfg/jungfrau4M/azav__azav_qbins']
    return q, azav_sum

def load_theory(theory_file):
    # Load theory file data
    split_tup = os.path.splitext(theory_file)
    filetype = split_tup[1]
    if filetype == '.txt':
        data = np.loadtxt(theory_file)
    elif filetype == '.npy':
        data = np.load(theory_file)
        data = np.transpose(data)
        print('npy data:'); print(data)
        print(data.shape)
    q_theory = data[0]
    f_theory = data[1]
    return q_theory, f_theory

def interp_theory(q_exp, q_theory, f_theory):
    # interpolate to shape of exp data
    interpolated_values = np.interp(q_exp, q_theory, f_theory)  
    return interpolated_values

def get_irf(run_numbers, blank_subtraction, theory_file, q_exp, indexstart):
    '''
    Returns the instrument response function (IRF) for calibration run(s)
    '''
    # Get azav for run(s)
    q_not_used, azav = get_azav(run_numbers)
    # Subtract blank
    azav_subtracted = azav - blank_subtraction
    # interpolation (make theory curve same shape as experiment q-bins)
    q_theory, f_theory = load_theory(theory_file)
    q_exp = q_exp[indexstart:]
    azav_subtracted = azav_subtracted[indexstart:]
    theory_interp = interp_theory(q_exp, q_theory, f_theory)
    #plt.plot(q_theory, ne_theory)
    #plt.plot(q_exp, ne_theory_interp)
    # Normalise before creating IRF
    exp_normalised = normalise(q_exp, azav_subtracted)
    theory_normalised = normalise(q_exp, theory_interp)
    tmp = theory_normalised / exp_normalised
    # convert nan and inf to 0
    irf = np.nan_to_num(tmp, copy=True, posinf=0, neginf=0)
    return irf

def irf_blank_correction(azav, azav_blank, irf):
    '''
    Applies blank subtraction and IRF correction to an azav curve
    '''
    print('azav_blank:'); print(azav_blank)
    print('irf:'); print(irf)
    azav_corrected = (azav - azav_blank) * irf
    return azav_corrected
# END THOMAS FUNCTIONS
###

# BLANK RUN
q, azav_blank = get_azav(blank_run_numbers)
# END BLANK RUN
###

# Neon and IRF
tmp, ne_azav = get_azav(ne_run_numbers)
q_theory, ne_theory = load_theory(ne_theory_file)
indexstart = 2
irf = get_irf(ne_run_numbers, azav_blank, ne_theory_file, q, indexstart)
ne_exp_corrected = irf_blank_correction(ne_azav, azav_blank, irf)
plt.figure()
plt.plot(q, normalise(q, irf))
plt.plot(q, normalise(q, ne_azav))
scale = np.max(normalise(q, ne_exp_corrected)) / np.max(normalise(q_theory, ne_theory))
print('Ne theory scale = %f' % scale)
plt.plot(q_theory, scale*normalise(q_theory, ne_theory)) 
plt.plot(q, normalise(q, ne_exp_corrected))
plt.legend(['IRF', 'Ne experiment', 'Ne theory', 'Ne exp corrected'])

# END Neon IRF
###

# SF6 sanity check
q_theory, sf6_theory = load_theory(sf6_theory_file)
sf6_theory_normalised = normalise(q_theory, sf6_theory)
tmp, sf6_azav = get_azav(sf6_run_numbers)
sf6_azav_corrected = irf_blank_correction(sf6_azav, azav_blank, irf)
sf6_exp_normalised = normalise(q, sf6_azav_corrected)

plt.figure()
scale = np.max(sf6_exp_normalised) / np.max(sf6_theory_normalised)
print('SF6 theory scale = %f' % scale)
A = 3.1
plt.plot(q_theory, A * scale * sf6_theory_normalised)
#print('sf6 azav corrected:')
#print(sf6_azav_corrected)
start = 5
plt.plot(q[start:], sf6_exp_normalised[start:], '.')
plt.xlim([0, 7])
plt.ylim([0, 0.1])
plt.legend(['SF6 theory', 'SF6 exp corrected'])

# END SF6 sanity check
###

print('Vector q ='); print(q)
#print('Size of qbin increment: %f (A-1)' % qbin)
#print('qbins:'); print(qbins)
#print(x)
#print(z)

/cds/data/drpsrcf/cxi/cxilx9320/scratch/hdf5/smalldata/cxilx9320_Run0051.h5, FILE EXISTS, CONTINUE...
azav shape: 
(36922, 35)
/cds/data/drpsrcf/cxi/cxilx9320/scratch/hdf5/smalldata/cxilx9320_Run0052.h5, FILE EXISTS, CONTINUE...
azav shape: 
(51767, 35)
/cds/data/drpsrcf/cxi/cxilx9320/scratch/hdf5/smalldata/cxilx9320_Run0052.h5, FILE EXISTS, CONTINUE...
azav shape: 
(51767, 35)
azav_blank:
[ 0.         45.88414824 45.61767424 46.1250131  41.51331354 37.47697747
 33.59247584 29.54214625 25.70765775 22.19292579 19.2813566  16.55105008
 14.91663906 13.61967635 11.74152437 10.46183459  9.71072751  8.9479021
  8.54934325  8.14005307  7.87578099  7.87281265  7.53296748  7.33053469
  7.11788128  7.7303047   7.16849776  6.89561744  6.87142522  6.74673971
  6.89301082  6.75962607  6.76284605  6.67290836  6.33082746]
irf:
[1.11994208 1.03325187 1.0360032  1.03424948 1.03343949 1.03075833
 1.02415787 1.01817664 1.02188446 1.02738992 1.0208765  1.0144064
 1.0063406  0.99919899 0.98854491 0.9866362

ValueError: operands could not be broadcast together with shapes (35,) (33,) 